# Overview:
 - we processed the data to have the same measure for all studies 
 - we chose OR as the common measure for all studies
 - we also chose to use the 95% CI for all studies

# Steps:
1. read data
2. cleaned up file to keep only the numbers
   * removed all the special formatting
   * changed column names to simplify
3. converted all the measures to OR
   * cut the ORs to be only between `0` and `10` because in log OR we can not have anything less than 0 
   * aka: if there were any numbers less than `0.1`, they were given the value `0.1`
   * aka: if there were any numbers greater than `10`, they were given the value `10`
   This is because the `Crude IRR` conversion could result in infinity
   (when dividing by 0, will give a result of infinity-- not useful for us)
      ```R
      if (measure_name == "Crude IRR"){
         OR <- measure / (1 - measure)
      }
      ```

In [1]:
# Function to convert measures to OR
convert_to_or <- function(row){

  measure_name <- row[['Measure_Name']]
  measure <- as.numeric(row[['Measure_Value']])
  n1 <- as.numeric(row[['N1_Cancer']])
  n2 <- as.numeric(row[['N2_No_Cancer']])

  if (measure_name == "OR"){
    OR <- measure
  } else if (measure_name == "Crude OR") {
    OR <- measure
  } else if (measure_name == "Adjusted OR") {
    OR <- measure
  } else if (measure_name == "RD"){
    OR <- measure
  } else if (measure_name == "SIR"){
    # TODO: two SIR values are 0, please fix the upstream data
    OR <- measure
  } else if (measure_name == "RR"){
    OR <- measure
  } else if (measure_name == "HR"){
    OR <- measure
  } else if (measure_name == "Crude IRR"){
    OR <- measure
  } else if (measure_name == "AHR"){
    OR <- measure 
  }
  # print(OR)
  if (OR < 0.1) {
    OR <- 0.0001
  } else if (OR > 10) {
    OR <- 10
  }
  return(OR)
}

In [2]:
# read data in
data <- read.csv("studies-raw-data.csv")

# remove commas from Total_Cohort and parse as numeric
data$n <- as.numeric(gsub(",", "",data$Total_Cohort))

# convert Measure_Value to numeric
data$Measure_Value <- as.numeric(data$Measure_Value)

# parse N1_Cancer as numeric or fill with 0
data$N1_Cancer <- as.numeric(data$N1_Cancer)
data$N1_Cancer[is.na(data$N1_Cancer)] <- 0

# parse N2_No_Cancer as numeric or fill with 0
data$N2_No_Cancer <- as.numeric(data$N2_No_Cancer)
data$N2_No_Cancer[is.na(data$N2_No_Cancer)] <- 0

# delete rows with no confidence interval
data <- data[!is.na(data$X95_CI_lower),]
data <- data[!is.na(data$X95_CI_upper),]

# show the first few rows
head(data)

,Article_ID,Total_Cohort,Measure_Name,Measure_Value,X95_CI_lower,X95_CI_upper,N1_Cancer,N2_No_Cancer,Cancer.Type,Exclude.From.Overall.Plot,CANCER.GROUP,X,n
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<dbl>
1,"Di et al. Lucca, 2010","1,216",OR,3.04,1.22,7.57,0,0,Melanoma,,SKIN,NA,1216
2,"Brewster et al., 2010","77,518",SIR,1.40,0.17,5.04,0,0,Melanoma,,SKIN,NA,77518
3,"Brewster et al., 2010","77,518",SIR,0.00,0.00,3.11,0,0,Basal Cell Carcinoma,,SKIN,NA,77518
4,"Cnattingius et al., 1995",98,OR,7.50,1.80,31.90,0,0,Myeloid Leukemia,,BLOOD,NA,98
5,"Bugaiski et al.-Shaked, 2022","342,172",HR,1.33,0.95,1.84,0,0,Total Malignant Morbidity,,SOLID ORGAN,NA,342172
6,"Bugaiski et al.-Shaked, 2022","342,172",HR,1.09,0.92,1.30,0,0,Total Bengin Tumors,,SOLID ORGAN,NA,342172


In [3]:
# convert to OR
data$OR <- apply(data, 1, convert_to_or)

# show the first few rows
head(data)

,Article_ID,Total_Cohort,Measure_Name,Measure_Value,X95_CI_lower,X95_CI_upper,N1_Cancer,N2_No_Cancer,Cancer.Type,Exclude.From.Overall.Plot,CANCER.GROUP,X,n,OR
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>
1,"Di et al. Lucca, 2010","1,216",OR,3.04,1.22,7.57,0,0,Melanoma,,SKIN,NA,1216,3.0400
2,"Brewster et al., 2010","77,518",SIR,1.40,0.17,5.04,0,0,Melanoma,,SKIN,NA,77518,1.4000
3,"Brewster et al., 2010","77,518",SIR,0.00,0.00,3.11,0,0,Basal Cell Carcinoma,,SKIN,NA,77518,0.0001
4,"Cnattingius et al., 1995",98,OR,7.50,1.80,31.90,0,0,Myeloid Leukemia,,BLOOD,NA,98,7.5000
5,"Bugaiski et al.-Shaked, 2022","342,172",HR,1.33,0.95,1.84,0,0,Total Malignant Morbidity,,SOLID ORGAN,NA,342172,1.3300
6,"Bugaiski et al.-Shaked, 2022","342,172",HR,1.09,0.92,1.30,0,0,Total Bengin Tumors,,SOLID ORGAN,NA,342172,1.0900


In [4]:
# rename or to odds_ratio
data$odds_ratio <- data$OR

# rename X95_CI_lower to lower_ci
data$lower_ci <- data$X95_CI_lower

# rename X95_CI_upper to upper_ci
data$upper_ci <- data$X95_CI_upper

# rename X...Article_ID to article_id
data$article_id <- data$Article_ID

# rename Cancer.Type to cancer_type
data$cancer_type <- data$Cancer.Type

# rename CANCER.GROUP tp cancer_group
data$cancer_group <- data$CANCER.GROUP

# rename Exclude.From.Overall.Plot to exclude_from_overall_plot
data$exclude_from_overall_plot <- data$Exclude.From.Overall.Plot

# First char and last two chars of Article_ID
data$Article_Code <- paste(substr(data$Article_ID, 1, 1), substr(data$Article_ID, nchar(data$Article_ID)-1, nchar(data$Article_ID)), sep="")

# show the first few rows
head(data)

,Article_ID,Total_Cohort,Measure_Name,Measure_Value,X95_CI_lower,X95_CI_upper,N1_Cancer,N2_No_Cancer,Cancer.Type,Exclude.From.Overall.Plot,...,n,OR,odds_ratio,lower_ci,upper_ci,article_id,cancer_type,cancer_group,exclude_from_overall_plot,Article_Code
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,"Di et al. Lucca, 2010","1,216",OR,3.04,1.22,7.57,0,0,Melanoma,,...,1216,3.0400,3.0400,1.22,7.57,"Di et al. Lucca, 2010",Melanoma,SKIN,,D10
2,"Brewster et al., 2010","77,518",SIR,1.40,0.17,5.04,0,0,Melanoma,,...,77518,1.4000,1.4000,0.17,5.04,"Brewster et al., 2010",Melanoma,SKIN,,B10
3,"Brewster et al., 2010","77,518",SIR,0.00,0.00,3.11,0,0,Basal Cell Carcinoma,,...,77518,0.0001,0.0001,0.00,3.11,"Brewster et al., 2010",Basal Cell Carcinoma,SKIN,,B10
4,"Cnattingius et al., 1995",98,OR,7.50,1.80,31.90,0,0,Myeloid Leukemia,,...,98,7.5000,7.5000,1.80,31.90,"Cnattingius et al., 1995",Myeloid Leukemia,BLOOD,,C95
5,"Bugaiski et al.-Shaked, 2022","342,172",HR,1.33,0.95,1.84,0,0,Total Malignant Morbidity,,...,342172,1.3300,1.3300,0.95,1.84,"Bugaiski et al.-Shaked, 2022",Total Malignant Morbidity,SOLID ORGAN,,B22
6,"Bugaiski et al.-Shaked, 2022","342,172",HR,1.09,0.92,1.30,0,0,Total Bengin Tumors,,...,342172,1.0900,1.0900,0.92,1.30,"Bugaiski et al.-Shaked, 2022",Total Bengin Tumors,SOLID ORGAN,,B22


In [5]:
# save data into a new csv
write.csv(data, "processed-data.csv")